In [1]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [2]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Test_Open_CV'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Mounted at /content/drive


['models', 'FaceImage', 'data_face_features.pickle']

In [3]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/Test_Open_CV/data_face_features.pickle", mode="rb"))

In [4]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.11157133  0.12669042 -0.14826356 ... -0.04966913  0.07908837
   -0.04749148]]

 [[ 0.12540211  0.12688084 -0.13584705 ... -0.07767584  0.04618051
   -0.04827663]]

 [[ 0.13139696  0.13920398 -0.1188855  ... -0.09052998  0.05804444
   -0.04813231]]

 ...

 [[ 0.05476612  0.19814469 -0.02060484 ... -0.06942512  0.11944059
    0.01252593]]

 [[ 0.07348809  0.17351446 -0.01863037 ... -0.07232255  0.09691378
    0.00313703]]

 [[ 0.05880686  0.1767127  -0.02341492 ... -0.05536838  0.07932062
    0.00897073]]]
['162000309_LeHoangQuan' '162000309_LeHoangQuan' '162000309_LeHoangQuan'
 ... '1721031695_DinhDuyLong' '1721031695_DinhDuyLong'
 '1721031695_DinhDuyLong']


In [5]:
x.shape

(4714, 1, 128)

In [6]:
y.shape

(4714,)

In [7]:
x = x.reshape(-1, 128)

In [8]:
x.shape

(4714, 128)

In [9]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [10]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3771, 128), (943, 128), (3771,), (943,))

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [12]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [13]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [14]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.71
Accurency Test = 0.68
F1 Score Train = 0.50
F1 Score Test = 0.48


In [15]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [16]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.77
F1 Score Train = 1.00
F1 Score Test = 0.66


In [17]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [18]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [19]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.83
F1 Score Train = 0.97
F1 Score Test = 0.73


In [20]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [21]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   5.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   5.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   5.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   5.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.2s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [22]:
model_best_estimator = model_grid.best_estimator_

In [23]:
model_grid.best_score_

0.8483160965261204

In [24]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/Test_Open_CV/ml_face_person_identity.pkl", mode='wb'))